In [ ]:
import requests
import pandas as pd
import math
import os
from scrapy import Selector
import csv

os.makedirs('book_data', exist_ok=True)
os.makedirs('book_images', exist_ok=True)

category_files = {}
category_writers = {}

i = 1

while i < 51:
    url_pages = f"https://books.toscrape.com/catalogue/page-{i}.html"
    url = "https://books.toscrape.com"
    response = requests.get(url_pages)

    sel = Selector(text=response.text)
    articles = sel.css("article.product_pod")

    for article in articles:
        title = article.css("h3 a::text").get()
        price = article.css("div.product_price p.price_color::text").get()
        stock = article.css("div.product_price p.availability::text")[1].get().strip()
        image = article.css("div.image_container img::attr(src)").get()
        product = article.css("div.image_container a::attr(href)").get()

        url_second_page = url + "/catalogue/" + product
        response2 = requests.get(url_second_page)
        sel2 = Selector(text=response2.text)

        rows = sel2.css("tr")
        for row in rows[:1]:
            upc = row.css("td::text").getall()

        category = sel2.css("ul.breadcrumb a[href]::text")[2].get()

        clean_category = category.replace(" ", "_").replace("/", "_").replace("\\", "_")

        category_image_dir = f"book_images/{clean_category}"
        os.makedirs(category_image_dir, exist_ok=True)

        if clean_category not in category_files:
            csv_file = open(f'book_data/{clean_category}.csv', 'w', newline='', encoding='utf-8')
            writer = csv.writer(csv_file)
            # Write header - NOW INCLUDES RATING
            writer.writerow(['Title', 'Price', 'Stock', 'UPC', 'Category', 'Rating', 'Image_Path', 'Product_URL'])
            category_files[clean_category] = csv_file
            category_writers[clean_category] = writer

        image_url = url + "/" + image
        image_response = requests.get(image_url)

        upc_str = upc[0] if upc else "unknown"
        image_filename = f"{upc_str}.jpg"
        image_path = os.path.join(category_image_dir, image_filename)

        with open(image_path, 'wb') as f:
            f.write(image_response.content)

        rating_class = sel2.css("p.star-rating::attr(class)").get()
        rating_text = "Unknown"
        rating_numeric = 0

        if 'One' in rating_class:
            rating_text = 'One Star / Five'
            rating_numeric = 1
        elif 'Two' in rating_class:
            rating_text = 'Two Stars / Five'
            rating_numeric = 2
        elif 'Three' in rating_class:
            rating_text = 'Three Stars / Five'
            rating_numeric = 3
        elif 'Four' in rating_class:
            rating_text = 'Four Stars / Five'
            rating_numeric = 4
        elif 'Five' in rating_class:
            rating_text = 'Five Stars / Five'
            rating_numeric = 5

        writer = category_writers[clean_category]
        writer.writerow([
            title,
            price,
            stock,
            upc[0] if upc else "",
            category,
            rating_numeric,
            image_path,
            url_second_page
        ])

        print(f"Title: {title}")
        print(f"Price: {price}")
        print(f"Stock: {stock}")
        print(f"Rating: {rating_text} ({rating_numeric}/5)")
        print(f"Category: {category}")
        print(f"Image saved: {image_path}")
        print(f"UPC: {upc[0] if upc else 'Unknown'}")
        print("-" * 50)

    i = i + 1

for file in category_files.values():
    file.close()

print("Scraping completed! Check the 'book_data' and 'book_images' folders.")